<a href="https://colab.research.google.com/github/luizdevfelipe/gemini-api/blob/main/fazendo_consultas_com_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instalando a biblioteca do Gemini**

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 5.9 MB/s eta 0:00:00


**Importando as bibliotecas e definindo o valor da chave de api**

In [ ]:
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY = "..."
genai.configure(api_key=GOOGLE_API_KEY)

**Listando as opções de Embedding disponíveis atualmente**

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


**Gerando o Embedding de um texto**

In [ ]:
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
  "\n"
  "Artigo Completo:\n"
  "Gemini API &  Google IA Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa"
)

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT"
                                )
print(embeddings)

{'embedding': [0.057924394, -0.025613535, -0.02905703, 0.015980126, 0.070118986, -0.00544783, -0.03850862, -0.021485824, 0.061148725, 0.059253316, 0.009979207, 0.0177078, -0.0514286, -0.025866162, 0.012766809, -0.010350732, 0.029665383, -0.021517195, -0.02858633, -0.0053529087, 0.0011328751, 0.01115765, -0.037717417, -0.068905614, -0.01301938, 0.026069468, 0.010290482, -0.04169418, -0.032155544, 0.029071623, -0.051510803, 0.04768575, -0.03400309, 0.019855704, -0.04635761, -0.029827612, -0.03281578, -0.04932401, -0.0037273732, 0.005811202, 0.0033655253, -0.08310844, -0.012970934, 0.031352203, -0.0019298407, -0.026808398, 0.05243602, 0.049039952, 0.021144219, -0.055634256, 0.019500356, 0.019670853, 0.055746213, -0.04409391, -0.009455001, -0.0033930053, 0.020127703, -0.038823564, 0.019485408, -0.009411695, 0.00760206, 0.024597468, -0.0074767396, 0.05574909, 0.026598394, -0.06344206, -0.03990161, -0.0017180463, 0.00877865, 0.052883863, 0.003733146, 0.013279548, 0.050149173, -0.06651841, -0

**Gerando alguns textos com títulos e organizando em uma mesma lista**

In [ ]:
DOCUMENT1 = {
    "Título": "Operações do sistema de controle climático",
    "Conteúdo": "Sensores estrategicamente posicionados dentro do Google Car monitoram constantemente a temperatura, a umidade e a qualidade do ar, transmitindo esses dados para um controlador inteligente que, por sua vez, ajusta automaticamente o ar condicionado, a ventilação e a iluminação, garantindo um ambiente interno sempre agradável e personalizado, como um maestro conduzindo uma sinfonia de conforto.", }

DOCUMENT2 = {
    "Título":"Touchscreen",
    "Conteúdo":"A tela touchscreen do Google Car é como uma tela mágica que responde ao toque dos dedos, transformando comandos em ações intuitivas, permitindo navegar pelos menus, controlar o sistema de entretenimento, ajustar o clima e até mesmo visualizar informações importantes sobre a viagem, tudo com a fluidez e precisão de um maestro conduzindo uma orquestra digital.", }

DOCUMENT3 = {
    "Título":"Mudança de Marchas",
    "Conteúdo":"A caixa de câmbio do Google Car, totalmente automatizada, realiza a troca de marchas com a precisão de um relojoeiro, adaptando-se às condições da estrada e ao estilo de condução, garantindo uma experiência suave e eficiente, como um maestro conduzindo uma sinfonia de movimentos sincronizados." }

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

**Transformando esses textos gerados para o formato DataFrame através da biblioteca Pandas**

In [ ]:
df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']
df

,Titulo,Conteudo
0,Operações do sistema de controle climático,Sensores estrategicamente posicionados dentro ...
1,Touchscreen,A tela touchscreen do Google Car é como uma te...
2,Mudança de Marchas,"A caixa de câmbio do Google Car, totalmente au..."


**Criando uma função para gerar embeddings como colunas em um df**

In [ ]:
model = "models/embedding-001"

def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]


**Adicionando no DataFrame a coluna de embedding respectiva ao título e conteúdo informados**

In [ ]:
df['embedding'] = df.apply(lambda row: embed_fn(row['Titulo'], row['Conteudo']), axis=1)
df

,Titulo,Conteudo,embedding
0,Operações do sistema de controle climático,Sensores estrategicamente posicionados dentro ...,"[-0.008774455, -0.033588186, -0.019025063, 0.0..."
1,Touchscreen,A tela touchscreen do Google Car é como uma te...,"[0.009426177, -0.03511878, -0.012367844, -0.00..."
2,Mudança de Marchas,"A caixa de câmbio do Google Car, totalmente au...","[0.032781474, -0.033281382, -0.018694708, 0.03..."


**Função para gerar embedding do texto de consulta e retornar o conteúdo mais próximo da busca feita**

In [ ]:
def gerar_e_fazer_pesquisa(consult, base):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consult,
                                 task_type="RETRIEVAL_QUERY")['embedding']

  produtos_escalares = np.dot(np.stack(df['embedding']), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]['Conteudo']

**Fazendo uma busca no dataFrame e exibindo o valor do conteúdo mais próximo**

In [ ]:
trecho = gerar_e_fazer_pesquisa("Como funciona a troca de marchas do carro?", df)
print(trecho)

A caixa de câmbio do Google Car, totalmente automatizada, realiza a troca de marchas com a precisão de um relojoeiro, adaptando-se às condições da estrada e ao estilo de condução, garantindo uma experiência suave e eficiente, como um maestro conduzindo uma sinfonia de movimentos sincronizados.


**Fazendo o Gemini gerar uma resposta baseada naquilo que ele encontrar semelhante na base**

In [ ]:
prompt = f"Reescreva esse texto sem adicionar infromações que não façam parte do texto original: {trecho}"

generation_configs = {
    "candidate_count": 1,
    "temperature": 0.5,
}

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_configs)
response = model_2.generate_content(prompt)
print(response.text)

Imagina o Google Car como um carro que troca de marchas igualzinho a um ninja, na hora certa e sem nem suar a camisa. Ele se adapta a qualquer estrada, como se fosse um camaleão, e ao seu estilo de dirigir, como se fosse seu melhor amigo. É como uma música que toca sozinha, suave e certinha, sem você precisar nem encostar no rádio!
